# Analyzing the Spectrum of Explainability in Outcome-Oriented Predictive Process Monitoring

## traffic_fines_1

### data preprocessing

#### importing libraries and terminology

In [ ]:
#import libraries

In [ ]:
import pandas as pd
import numpy as np
import time
import os
import random
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tabulate import tabulate
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion
import xgboost as xgb

In [ ]:
#user specified packages from Teinemaa
from DatasetManager import DatasetManager

In [ ]:
#tree packages
import graphviz
from sklearn import tree

#LSTM
import numpy as np
import pandas as pd
from pandas import Series
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import logging
plt.style.use('fivethirtyeight')

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Flatten, Embedding, LSTM, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_json

from collections import OrderedDict
from collections import defaultdict

import tensorflow.keras
import tensorflow as tf 
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Input, Reshape, Activation, Multiply, TimeDistributed, concatenate, BatchNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import OrdinalEncoder

Terminology

In [ ]:
params_dir = './params_dir'
results_dir = './results' 
cls_encoding = 'agg'

gap = 1
n_iter = 1

bucket_encoding = "agg"

dataset_ref_to_datasets = {
    "bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(1,6)],
    "insurance": ["insurance_activity", "insurance_followup"],
    "sepsis_cases": ["sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4"]
}

encoding_dict = {
    "laststate": ["static", "last"],
    "agg": ["static", "agg"],
    "index": ["static", "index"],
    "combined": ["static", "last", "agg"],
    "onehot": ["static", "onehot"]
}

In [ ]:
#terminology

In [ ]:
dataset_ref = 'traffic_fines_1'

In [ ]:
datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]
methods = encoding_dict[cls_encoding]
    
train_ratio = 0.8
random_state = 22

In [ ]:
# print dataset name

In [ ]:
for dataset_name in datasets:
    print('Dataset:', dataset_name)

In [ ]:
# read the data

In [ ]:
dataset_manager = DatasetManager(dataset_name)

In [ ]:
data = dataset_manager.read_dataset()
cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                        'static_cat_cols': dataset_manager.static_cat_cols,
                        'static_num_cols': dataset_manager.static_num_cols, 
                        'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                        'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                        'fillna': True}

In [ ]:
#terminology

In [ ]:
params_dir = './params_dir'
results_dir = './results' 
DT_dir = './DT_dir'
cls_encoding = 'agg'

gap = 1
n_iter = 1

dataset_ref_to_datasets = {
    "bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(1,6)],
    "insurance": ["insurance_activity", "insurance_followup"],
    "sepsis_cases": ["sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4"]
}

encoding_dict = {
    "laststate": ["static", "last"],
    "agg": ["static", "agg"],
    "index": ["static", "index"],
    "combined": ["static", "last", "agg"],
    "onehot": ["static", "onehot"]
}

train_ratio = 0.8
random_state = 22

In [ ]:
# determine min and max (truncated) prefix lengths

In [ ]:
min_prefix_length = 1
if "traffic_fines" in dataset_name:
    max_prefix_length = 10
elif "bpic2017" in dataset_name:
    max_prefix_length = min(20, dataset_manager.get_pos_case_length_quantile(data, 0.90))
else:
    max_prefix_length = min(40, dataset_manager.get_pos_case_length_quantile(data, 0.90))

In [ ]:
max_prefix_length

#### data preprocessing

In [ ]:
# split into training and test
train, test = dataset_manager.split_data_strict(data, train_ratio, split="temporal")

In [ ]:
#prefix generation of train and test data

In [ ]:
print('prefixing started')
dt_train_prefixes = dataset_manager.generate_prefix_data(train, min_prefix_length, max_prefix_length)
dt_test_prefixes = dataset_manager.generate_prefix_data(test, min_prefix_length, max_prefix_length)

In [ ]:
dt_train_prefixes_original = dt_train_prefixes.copy()
dt_test_prefixes_original = dt_test_prefixes.copy()

In [ ]:
#get the label of the train and test set
test_y = dataset_manager.get_label_numeric(dt_test_prefixes)
train_y = dataset_manager.get_label_numeric(dt_train_prefixes)   

In [ ]:
auc_totals = []
preds_all = []
test_y_all = []
nr_events_all = []
offline_total_times = []
online_event_times = []

In [ ]:
nr_events_all.extend(list(dataset_manager.get_prefix_lengths(dt_test_prefixes)))  
test_y_all.extend(test_y)

### LSTM

#### poging 5

In [ ]:
#functions

def prepare_inputs(X_train, X_test, data):    
    oe = OrdinalEncoder()
    oe.fit(data)
    X_train_enc = oe.transform(X_train)
    X_test_enc = oe.transform(X_test)
    return X_train_enc, X_test_enc

def numeric_padding(sequences, maxlen=None, value=0):
    num_samples = len(sequences)
    sample_shape = np.asarray(sequences[0]).shape[1:]
    x = np.full((num_samples, maxlen) + sample_shape, value)
    for idx, s in enumerate(sequences):
        trunc = s[:maxlen]
        x[idx, :maxlen] = trunc[0]
        
def normalist(lst):
    s = sum(lst)
    norm = [float(i)/s for i in lst]
    return norm

    return x   

In [ ]:
#TRANSFORMATION

#categorical columns integercoded

dt_train_prefixes.dtypes

cat_columns = ['Activity']

dt_train_prefixes[cat_columns],dt_test_prefixes[cat_columns]= prepare_inputs(dt_train_prefixes[cat_columns], dt_test_prefixes[cat_columns], data[cat_columns])



dt_train_prefixes[cat_columns] = dt_train_prefixes[cat_columns]+1

dt_test_prefixes[cat_columns] = dt_test_prefixes[cat_columns]+1

#make embeddings

dt_train_prefixes.dtypes

cat_columns.append('Case ID')
cat_columns.append('label')

cat_columns

ans_train = [pd.DataFrame(y) for x, y in dt_train_prefixes[cat_columns].groupby('Case ID', as_index=False)]
ans_test = [pd.DataFrame(y) for x, y in dt_test_prefixes[cat_columns].groupby('Case ID', as_index=False)]

train_labels = []
for i in range (0,len(ans_train)):
    temp_label = ans_train[i]['label'].iloc[0]
    train_labels.append(temp_label)

test_labels = []
for i in range (0,len(ans_test)):
    temp_label = ans_test[i]['label'].iloc[0]
    test_labels.append(temp_label)

train_y = [1 if i!='regular' else 0 for i in train_labels]
test_y = [1 if i!='regular' else 0 for i in test_labels]

cat_columns.remove('label')
cat_columns.remove('Case ID')

cat_columns

maxlen = max_prefix_length

In [ ]:
#cutoff: max amount of events in a prefix
cutoff = maxlen
no_activities= len(data.groupby(['Activity']))+1

In [ ]:
#padding of the different categorical columns
#train paddings
paddings_train = []
for i in cat_columns:
    padding= []
    for k in range(0,len(ans_train)):
        temp = []
        temp = list(ans_train[k][i])
        padding.append(temp)
    
    padded = np.array(pad_sequences(padding,maxlen=maxlen, padding='pre', truncating='pre',value=0))
    padded = padded/len(data.groupby([i]))
    paddings_train.append(padded)

#test paddings
paddings_test = []
for i in cat_columns:
    padding= []
    for k in range(0,len(ans_test)):
        temp = []
        temp = list(ans_test[k][i])
        padding.append(temp)
    
    padded = np.array(pad_sequences(padding,maxlen=maxlen, padding='pre', truncating='pre',value=0))
    padded = padded/len(data.groupby([i]))
    paddings_test.append(padded)

In [ ]:
##### numerical columns
#NUMERICAL COLUMNS
numerical_columns = ['timesincelastevent',
'Case ID']

dt_train_prefixes2 = dt_train_prefixes[numerical_columns]
dt_test_prefixes2 = dt_test_prefixes[numerical_columns]

ans_train2 = [pd.DataFrame(y) for x,y in dt_train_prefixes2.groupby('Case ID', as_index=False)]
ans_test2 = [pd.DataFrame(y) for x,y in dt_test_prefixes2.groupby('Case ID', as_index=False)]

In [ ]:
pad_train = []
pad_test  = []
num_columns = ['timesincelastevent']
for i in num_columns:
    padding = []
    for k in range(0,len(ans_train2)):
        temp_train = []
        temp_train = list(ans_train2[k][i])
        padding.append(temp_train)
        
    padded = np.array(pad_sequences(padding,maxlen=maxlen, padding='pre', truncating='pre',value=0))
    padded = padded/data[i].max()
    pad_train.append(padded)
for i in num_columns:
    padding = []
    for k in range(0,len(ans_test2)):
        temp_test = []
        temp_test = list(ans_test2[k][i])
        padding.append(temp_test)
        
    padded = np.array(pad_sequences(padding,maxlen=maxlen, padding='pre', truncating='pre',value=0))
    padded = padded/data[i].max()
    pad_test.append(padded)

#### LSTM model

In [ ]:
import os
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import Sequence
from tensorflow.keras.regularizers import l2
from tensorflow.keras.constraints import non_neg, Constraint
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve


from tensorflow.keras.layers import Input, Concatenate, Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Nadam, Adam, SGD, Adagrad
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Bidirectional,TimeDistributed
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Softmax, Lambda, Multiply, Dropout
from tensorflow.keras import backend
from tensorflow.keras.layers import GlobalAveragePooling1D
import tensorflow.keras.utils as ku

In [ ]:
def create_index(log_df, column):
    """Creates an idx for a categorical attribute.
    Args:
        log_df: dataframe.
        column: column name.
    Returns:
        index of a categorical attribute pairs.
    """
    temp_list = log_df[[column]].values.tolist()
    subsec_set = {(x[0]) for x in temp_list}
    subsec_set = sorted(list(subsec_set))
    alias = dict()
    for i, _ in enumerate(subsec_set):
        alias[subsec_set[i]] = i + 1
    return alias


ac_index = create_index(dt_train_prefixes_original, 'Activity')
ac_index['Start'] = 0
ac_index['End'] = len(ac_index)
index_ac = {v: k for k, v in ac_index.items()}

ac_weights = ku.to_categorical(sorted(index_ac.keys()), len(ac_index))

In [ ]:
dropout_input = 0.01
dropout_context=0.30
lstm_size_alpha=64
lstm_size_beta=64
print("Training prefix and variable attention model")

l2reg=0.001
allow_negative=False
incl_time = True 
incl_res = True
#Code Input
#categorical embeddings
input_layer_a = Input(shape=(cutoff,), name='activity_input')

time_input_layer = Input(shape=(cutoff,1), name='time_input')

########################################
#inputs_list = [ac_input]
   
#Calculate embedding for each code and sum them to a visit level
embedding_activity = Embedding(ac_weights.shape[0],
                            ac_weights.shape[1],
                            weights=[ac_weights],
                            input_length=no_activities,
                           name='ac_embedding')(input_layer_a)
dim =ac_weights.shape[1]


full_embs = embedding_activity

#Apply dropout on inputs
full_embs = Dropout(dropout_input)(full_embs)
time_embs = concatenate([full_embs, time_input_layer], name='allInp')

dim += 1

alpha = Bidirectional(LSTM(lstm_size_alpha, return_sequences=True),
                                    name='alpha')
beta = Bidirectional(LSTM(lstm_size_beta, return_sequences=True),
                                   name='beta')
alpha_dense = Dense(1, kernel_regularizer=l2(l2reg))
beta_dense = Dense(dim, activation='tanh', kernel_regularizer=l2(l2reg))

#Compute alpha, visit attention
alpha_out = alpha(time_embs)
alpha_out = TimeDistributed(alpha_dense, name='alpha_dense_0')(alpha_out)
alpha_out = Softmax(axis=1, name='alpha_softmax')(alpha_out)
#Compute beta, codes attention
beta_out = beta(time_embs)
beta_out = TimeDistributed(beta_dense, name='beta_dense_0')(beta_out)
#Compute context vector based on attentions and embeddings
c_t = Multiply()([alpha_out, beta_out, time_embs])
c_t = Lambda(lambda x: backend.sum(x, axis=1))(c_t)
        #Reshape to 3d vector for consistency between Many to Many and Many to One implementations
        #contexts = L.Lambda(reshape)(c_t)

#Make a prediction
contexts = Dropout(dropout_context)(c_t)
 
output_layer = Dense(1, activation='sigmoid', name='final_output')(contexts)

In [ ]:
model = Model(inputs=[input_layer_a, time_input_layer], outputs=output_layer)

In [ ]:
opt = Nadam(lr=0.0005, beta_1=0.9, beta_2=0.999,
                   epsilon=1e-08, schedule_decay=0.004, clipvalue=3)

model.compile(loss={'final_output':'binary_crossentropy'}, optimizer= opt)
    
model.summary()

In [ ]:
output_file_path = os.path.join('./', 'models/model_rd_'+'_{epoch:02d}-{val_loss:.2f}.h5')

In [ ]:
padded_time = np.reshape(pad_train[0], (len(pad_train[0]), cutoff, 1))
padded_time_test=  np.reshape(pad_test[0], (len(pad_test[0]), cutoff, 1))

model_checkpoint = ModelCheckpoint(output_file_path,
                                       monitor='val_loss',
                                       verbose=1,
                                       save_best_only=True,
                                       save_weights_only=False,
                                       mode='auto')

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.5,
                                   patience=2,
                                   verbose=0,
                                   mode='auto',
                                   min_delta=0.0001,
                                   cooldown=0,
                                   min_lr=0)
model_inputs = [paddings_train[0]]
model_inputs.append(padded_time)

In [ ]:
start = time.time()
model.fit(model_inputs,
              train_y,
              callbacks=[early_stopping, lr_reducer, model_checkpoint],
              validation_split = 0.1,
              verbose=1,
              epochs=10)
model_inputs_test = [paddings_test[0]]
model_inputs_test.append(padded_time_test)
predictions = model.predict(model_inputs_test)
auc = roc_auc_score(test_y,predictions)
total_time = time.time() - start
print(total_time)
print(auc)

In [ ]:
model_name = 'BPIC2017_PC10'
model_json = model.to_json()
with open('./models/'+model_name+".json", "w") as json_file:
            json_file.write(model_json)
            # serialize weights to HDF5
            model.save_weights('./models/'+model_name+".h5")
            print("Saved model to disk")
            print('Name: '+model_name)

In [ ]:
preds_all = list(predictions)
method_name = "attention"
cls_method = "LSTM"

In [ ]:
if gap > 1:
    outfile = os.path.join(results_dir, "performance_results_%s_%s_%s_gap%s.csv" % (cls_method, dataset_name, method_name, gap))
else:
    outfile = os.path.join(results_dir, "performance_results_%s_%s_%s.csv" % (cls_method, dataset_name, method_name))

In [ ]:
len(nr_events_all)

In [ ]:
len(predictions)

In [ ]:
with open(outfile, 'w') as fout:
        fout.write("%s;%s;%s;%s;%s;%s\n"%("dataset", "method", "cls", "nr_events", "metric", "score"))
        fout.write("%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method,-1, "runtime_total", total_time))
        fout.write("%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method,-1, "runtime_avg", total_time/10))  
        dt_results = pd.DataFrame({"actual": test_y_all, "predicted": preds_all, "nr_events": nr_events_all})
        for nr_events, group in dt_results.groupby("nr_events"):
            if len(set(group.actual)) < 2:
                fout.write("%s;%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method, nr_events,-1, "auc", np.nan))
            else:
                fout.write("%s;%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method, nr_events,-1, "auc", roc_auc_score(group.actual, group.predicted)))
        fout.write("%s;%s;%s;%s;%s;%s\n"%(dataset_name, method_name, cls_method,-1, "auc", roc_auc_score(dt_results.actual, dt_results.predicted)))
        fout.write("%s\n"%(auc))

In [ ]:
print('Opening model')
json_file = open('./models/'+model_name+'.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model2 = model_from_json(loaded_model_json)
# load weights into new model
print('Opening weights')
model2.load_weights('./models/'+model_name+".h5")
print("Loaded model from disk")

In [ ]:
#   Next event selection method and numbers of repetitions
#variants = [{'imp': 'Arg Max', 'rep': 1}]#,
               # {'imp': 'Random Choice', 'rep': 1}]
#   Generation of predictions
has_time=False
    #model = load_model(os.path.join(output_route, parameters['model_file']))
model = model
layer_names = [layer.name for layer in model.layers]
print(layer_names)
l_emb_weights=None
ac_emb_weights = model.get_layer(name='ac_embedding').get_weights()[0]

s_emb_weights = model.get_layer(name='s_embedding').get_weights()[0]
    
at_emb_weights = model.get_layer(name='at_embedding').get_weights()[0]  
    
acc_emb_weights = model.get_layer(name='acc_embedding').get_weights()[0]  
    
li_emb_weights = model.get_layer(name='li_embedding').get_weights()[0]  

has_time=True

ac_output_weights, ac_bias = model.get_layer(name='final_output').get_weights()
print(ac_output_weights)
model_with_attention = Model(model.inputs, model.outputs +\
                                              [model.get_layer(name='alpha_softmax').output,\
                                               model.get_layer(name='beta_dense_0').output])

In [ ]:
temporal_vectors = []
variable_vectors=[]
predictions = []
for i in range(0,len(paddings_test[0])):
    x_ac_ngram = paddings_test[0][i].reshape((1,cutoff))
    x_s_ngram = paddings_test[1][i].reshape((1,cutoff))
    x_at_ngram = paddings_test[2][i].reshape((1,cutoff))
    x_acc_ngram = paddings_test[3][i].reshape((1,cutoff))
    x_li_ngram = paddings_test[4][i].reshape((1,cutoff))
    x_in_ngram  = pad_creditscore_test[i].reshape(1, 40, 1)
    x_t_ngram   = padded_time_test[i].reshape(1, 40, 1)
    betas=None      
    proba, alphas, betas = model_with_attention.predict([x_ac_ngram, x_s_ngram, x_at_ngram, x_acc_ngram, x_li_ngram, x_in_ngram,x_t_ngram])
    #print(proba, alphas, betas)
    proba = np.squeeze(proba)
    alphas = np.squeeze(alphas)
    temporal_att_vec = alphas
    assert (np.sum(temporal_att_vec) - 1.0) < 1e-5
    #print(temporal_att_vec)
    temporal_vectors.append(temporal_att_vec)

    if betas is not None:
        #get the beta value
        betas = np.squeeze(betas)
        idx = np.argmax(alphas)
        #print(idx)
        beta_val = betas[idx]
        # get the activity and role for that idx
        act_ip = int(x_ac_ngram[0][idx])
        ac_emb = ac_emb_weights[act_ip]
        dim=ac_emb.shape[0]
        
        rol_ip = int(x_s_ngram[0][idx])
        s_emb = s_emb_weights[rol_ip]
        dim += s_emb.shape[0]
            
       
        rol_ip = int(x_at_ngram[0][idx])
        at_emb = at_emb_weights[rol_ip]
        dim += at_emb.shape[0]
        
        
        rol_ip = int(x_acc_ngram[0][idx])
        acc_emb = acc_emb_weights[rol_ip]
        dim += acc_emb.shape[0]
           
        
        rol_ip = int(x_li_ngram[0][idx])
        li_emb = li_emb_weights[rol_ip]
        dim += li_emb.shape[0]
            
        num_v = np.squeeze(x_in_ngram)[idx]
        dim +=1
        
        if(betas.shape[1]==dim+1):
            time_v = np.squeeze(x_t_ngram)[idx]  # time and role as masked together
            emb = np.concatenate((ac_emb,s_emb,at_emb,acc_emb,li_emb,num_v,time_v), axis=None)
          
        #print('beta_val',beta_val.shape)
        beta_scaled = np.multiply(beta_val,emb)
        variable_attn = alphas[idx] * beta_scaled
        #sum_grad = np.sum(ac_output_weights, axis=1)
        #variable_attn=np.multiply(sum_grad.flatten(), variable_attn)
        #if pos!=test_y[i]:
            #temporal_vectors[i] = np.zeros(40, dtype='float32')
            #variable_attn = np.zeros(52, dtype='float32')
            #variable_vectors.append(variable_attn)
        predictions.append(proba)
        variable_vectors.append(variable_attn)

In [ ]:
auc = roc_auc_score(test_y, predictions)
auc

In [ ]:
auc

In [ ]:
temp_final = np.mean(np.array(temporal_vectors), axis=0)
pd.DataFrame(temp_final, columns=['alpha attention weight']).plot(kind='bar',
                                                                        title='Attention of '
                                                                              ' index', figsize=(10,7))
# Hide grid lines
plt.grid(False)
plt.show()

In [ ]:
if(len(variable_vectors)>0):
    var_final = np.mean(np.array(variable_vectors), axis=0)
    ac_labels = [index_ac[key] for key in sorted(index_ac.keys())]
    s_labels = [index_s[key] for key in sorted(index_s.keys())]
    at_labels = [index_at[key] for key in sorted(index_at.keys())]
    acc_labels = [index_acc[key] for key in sorted(index_acc.keys())]
    li_labels = [index_li[key] for key in sorted(index_li.keys())]
    print(ac_labels)
    print(s_labels)
    print(at_labels)
    print(acc_labels)
    print(li_labels)
      
    num_dim = var_final.shape[0]
    print(num_dim)
      
    if s_emb_weights is not None:
        ac_labels.extend(s_labels)
    if at_emb_weights is not None:
        ac_labels.extend(at_labels)  
    if acc_emb_weights is not None:
        ac_labels.extend(acc_labels)
    if li_emb_weights is not None:
        ac_labels.extend(li_labels)
    ac_labels.append('CreditScore')
    ac_labels.append('time')
        
    df_var=pd.DataFrame({'attributes':var_final, 'attribute_values':ac_labels})
    print(df_var)
    df_var.plot.bar(y='attributes', x='attribute_values',
                                title='Attention of the event attributes.', figsize=(10,7))
                                                                               
    #plot_history( plt, file_name + 'variable_attn', path )
      

    # Hide grid lines
    plt.grid(False)
    plt.show()

In [ ]:
var_local = variable_vectors[200]
df_var_local=pd.DataFrame({'attributes':var_local, 'attribute_values':ac_labels})
df_var_local.plot.bar(y='attributes', x='attribute_values',
                          title='Attention of the event attributes.')
                                                                               
plt.show()
#plotting local explanation
local_var = temporal_vectors[19]
pd.DataFrame(local_var, columns=['alpha attention weight']).plot(kind='bar',
                                                                        title='Attention of '
                                                                               ' index')                                                             
plt.show()